In [241]:
import pandas as pd
from ete3 import Tree, NodeStyle
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path as P
from collections import defaultdict, Counter
from debug import debugger, saver, loader
from scipy import stats
from matplotlib import cm
import pickle
from tqdm import tqdm

In [244]:
ROOT = "../output/results/expLI_cbmos2021/"
ROOT_EXPORT = "../output/results/expLI_cbmos2021/export"

In [245]:
P(ROOT_EXPORT).mkdir(parents=True, exist_ok=True)

In [239]:
def extract_param_from_name(name):
    name = "_".join(name.split("_")[1:])
    name = ".".join(name.split(".")[:-1])
    shorten = name
    # shorten = name.replace("__", "_")
    
    exp, model, *param, value, n = shorten.split("_")
    param = "_".join(param)
    return dict(exp=exp, model=model, param=param, value=value, n=n, name=name)

In [7]:
def read_dir(root):
    all_params = list()
    for path, subdirs, files in os.walk(root):
        for file in files:
            if file.startswith("stats_"):
                all_params.append(extract_param_from_name(file))
                
    return all_params
params = read_dir(ROOT)

In [9]:
def gather_groups(params):
    groups = defaultdict(list)
    for param in params:
        key = tuple((k, v) for k, v in param.items() if k in ["exp", "model", "param", "value"])
        groups[key].append(param)
    return groups

def gather_metagroups(groups):
    metagroups = defaultdict(dict)
    controls = {listtuple_to_dict(key_group)["model"]: value_group for key_group, value_group in groups.items()
               if listtuple_to_dict(key_group)["param"] == ""}
    exp = {key_group: value_group for key_group, value_group in groups.items()
               if listtuple_to_dict(key_group)["param"] != ""}
    
    for key_group, value_group in exp.items():            
        key_meta = tuple((k, v) for k, v in key_group if k in ["exp", "model", "param"])
        value = {k: v for k, v in key_group}["value"]
        metagroups[key_meta][value] = value_group
        
    for key_meta in metagroups:
        model = listtuple_to_dict(key_meta)["model"]
        param = listtuple_to_dict(key_meta)["param"]
        defval = str(get_default_val(param))
        
        metagroups[key_meta][defval] = controls[model]
    
    return metagroups

In [11]:
groups = gather_groups(params)

metagroups = gather_metagroups(groups)

In [12]:
renderers = list()

In [13]:
def recursive_add(history, tree, cell_id):
    if cell_id == -1:
        return
    cell = history.loc[cell_id]
    node = tree.add_child(name=cell_id, dist=cell["Tc_h"])
    node.set_style(STYLE.get(cell["type"]))
    for child_id in cell[["child1", "child2"]]:
        recursive_add(history, node, child_id)

def tree_from_cell(history, cell_id):
    tree = Tree()
    recursive_add(history, tree, cell_id)
    return tree

In [119]:
# PREPROCESS
def preprocess_history(param):
    file = "history_" + param["name"] + ".csv"
    df = pd.read_csv(P(ROOT) / file, index_col=0)
    df["index"] = df["index"].astype(int)
    df["child1"] = df["child1"].fillna(-1)
    df["child2"] = df["child2"].fillna(-1)
    df["child1"] = df["child1"].astype(int)
    df["child2"] = df["child2"].astype(int)
    df.set_index("index")
    return df

def preprocess_stats(param):
    file = "stats_" + param["name"] + ".csv"
    df = pd.read_csv(P(ROOT) / file)
    return df
        
def preprocess_nb_progenitor(param):
    stats = preprocess_stats(param)
    ref = stats.whole_pop_size.iloc[0]
    return dict(x=stats.time, y=stats.progenitor_pop_size / ref)

def preprocess_nb_cells(param):
    stats = preprocess_stats(param)
    ref = stats.whole_pop_size.iloc[0]
    return dict(x=stats.time, y=stats.whole_pop_size / ref)

def preprocess_ratio(param):
    stats = preprocess_stats(param)
    stats = stats.fillna(0)
    if "size_type_Cycling" in stats.columns:
        non_IP = stats.size_type_Cycling
        IP = 0
    else:
        non_IP = stats.size_type_RG + stats.size_type_GP if "size_type_GP" in stats.columns else stats.size_type_RG
        IP = stats.size_type_IP
    
    return dict(x=stats.time, y=IP / (non_IP + IP))

In [261]:
def preprocess_progeny(param):
    history = preprocess_history(param)
    return progeny_along_time(history)

def preprocess_progeny_all(param):
    history = preprocess_history(param)
    return progeny_along_time(history, only_leaves=False)

In [259]:
def count_total_progeny(history, cell_id, tree=None, only_leaves=True):
    if tree is None:
        tree = tree_from_cell(history, cell_id)
    if only_leaves:
        nodes = tree.get_leaves()
    else:
        nodes = list(filter(lambda x: isinstance(x.name, (int, np.integer)), tree.traverse()))
        
    return Counter(list(map(lambda x: history.loc[x.name]["type"], nodes)))

def progeny_along_time(history, _type="RG", only_leaves=True, step=2., appear_time=49.):
    ret_df = pd.DataFrame()
    
    min_time, max_time = min(history["appear_time"]), max(history["appear_time"])
    dict_trees = dict()
    # build trees
    for i, row in history.iterrows():
        if row["appear_time"] == appear_time:
            tree = tree_from_cell(history, row["index"])
            dict_trees.update({x.name: x for x in tree.traverse()})
    
    for T in np.arange(min_time, max_time, step):
        count_T = Counter()
        df_T = history[(history["appear_time"] >= T) &
                  (history["appear_time"] < (T + step)) & (history["type"] == _type)]
        N = len(df_T)
        if not N:
            continue
            
        for i, cell in df_T.iterrows():
            progeny = count_total_progeny(history, cell["index"], dict_trees.get(cell["index"]), only_leaves=only_leaves)
            count_T.update(progeny)
            
        new_row = {k: v / N for k, v in dict(count_T).items()}
        new_row["time"] = T
        ret_df = ret_df.append(new_row, ignore_index=True)
        
    ret_df = ret_df.fillna(0.0)
        
    return ret_df

In [ ]:
# tree.render("mytree.png", w=183, units="mm")

In [260]:
def draw_sum(line):
    total = 0
    for k in sorted(set(line.keys()) - {"x", "label"}):
        plt.fill_between(line["x"], total, total + line[k]["mean"], label=k)
        total = total + line[k]["mean"]
    plt.legend()

def draw_line(x, y, label=None):
    if isinstance(y, dict):
        return draw_mean_sd_line(x, y["mean"], y["sd"], label=label)
        
    plt.plot(x, y, label=label)
        
def draw_mean_sd_line(x, mean, sd, label=None):
    plt.plot(x, mean, label=label)
    plt.fill_between(x, (mean-sd), (mean+sd), alpha=.3)

def plot_lines(lines, title=None):
    lines.sort(key=lambda x: x.get("label", "NO_LABEL"))
    plt.figure()
    for line in lines:
        draw_line(line["x"], line["y"], line.get("label"))
    plt.legend()
    if title:
        plt.title(title)

def plot_sum(lines, title=None):
    lines.sort(key=lambda x: x.get("label", "NO_LABEL"))
    if len(lines) > 6:
        raise ValueError("Maximum allowed plot is 6")
    plt.figure(figsize=(10, 12))
    for i, line in enumerate(lines):
        plt.subplot(3, 2, i + 1)
        draw_sum(line)
        plt.title(line.get("label", "NO_LABEL"))
    if title:
        plt.suptitle(title)

In [233]:
def plot_metrics(lines, title=None):
    plt.figure(figsize=(16, 12))
    keys = set().union(*[element.keys() for element in lines]) - {"label"}
    lines.sort(key=lambda x: x.get("label", "NO_LABEL"))
    labels = [element.get("label", "NO_LABEL") for i, element in enumerate(lines)]
    null = {"mean": 0, "sd": 0}
    for i, k in enumerate(keys):
        plt.subplot(3, 4, i + 1)
        plt.title(k)
        vec = [element.get(k, null).get("mean") for element in lines]
        err = [element.get(k, null).get("sd") for element in lines]
        plt.bar(labels, vec, yerr=err, color=cm.Set2.colors)
        
    if title:
        plt.suptitle(title)

In [161]:
def mean_sd(ls):
    min_len = min([len(sample["x"]) for sample in ls])
    x = ls[0]["x"][:min_len]
    mean = np.mean([sample["y"][:min_len] for sample in ls], axis=0)
    sd = np.std([sample["y"][:min_len] for sample in ls], axis=0)
    return {"x": x, "y":{"mean": mean, "sd": sd}}

def mean_progeny(ls):
    """Runs on all columns"""
    min_len = min([len(sample["time"]) for sample in ls])
    dict_res = dict(x=ls[0]["time"][:min_len])
    
    col = {colname for sample in ls for colname in sample.columns} - {"time"}

    for c in col:
        mean = np.mean([sample.get(c, [0] * min_len)[:min_len] for sample in ls], axis=0)
        sd = np.std([sample.get(c, [0] * min_len)[:min_len] for sample in ls], axis=0)
        dict_res[c] = {"mean": mean, "sd": sd}
    return dict_res

def mean_dict(ls):
    dict_res = dict()
    keys = set().union(*[element.keys() for element in ls])
    for k in keys:
        vec = [element[k] for element in ls if k in element]
        dict_res[k] = {"mean": np.mean(vec), "sd": np.std(vec)}
        
    return dict_res

In [159]:
def merge_parents(df):
    tmp_df = pd.merge(df, df, how='inner', left_on="child1", right_on="index", suffixes=('_M', '_D1'))
    full_df = pd.merge(tmp_df, df.rename(columns=lambda x: x + "_D2"), how='inner', left_on="child2_M",
                       right_on="index_D2", suffixes=('_M', '_D2'))
    full_df["group"] = 2 + 1 * (full_df["appear_time_M"] > 75)
    full_df
    return full_df

def get_sub_df_merged(full_df, key1, key2, group):
    no_gp = (full_df["type_D1"] != "GP") & (full_df["type_D2"] != "GP")
    notnull = full_df[key1].notnull() & full_df[key2].notnull()
    notzero = (full_df[key1] != 0.) & (full_df[key2] != 0.)
    goodgroup = full_df["group"].isin(group)
    filt =  goodgroup & no_gp & notnull & notzero
    cur_df = full_df.loc[filt, :]
    return cur_df

def metrics_tc_mother_daughter(full_df):
    key1, key2 = "Tc_h_M", "Tc_h_D1"
    cur_df = get_sub_df_merged(full_df, key1, key2, [2, 3])
    var1 = cur_df[key1] / cur_df[key2]
    var2 = cur_df[key1] - cur_df[key2]
    return dict(
        mean_ratio_m_d_g23=np.mean(var1), 
        std_ratio_m_d_g23=np.std(var1), 
        mean_diff_m_d_g23=np.mean(var2), 
        std_diff_m_d_g23=np.std(var2)
    )

def metrics_tc_daughters(full_df):
    key1, key2 = "Tc_h_D1", "Tc_h_D2"
    cur_df_g2 = get_sub_df_merged(full_df, key1, key2, [2])
    cur_df_g3 = get_sub_df_merged(full_df, key1, key2, [3])
    cur_df_g23 = get_sub_df_merged(full_df, key1, key2, [2, 3])
    return dict(
        corr_tc_daughter_g2=stats.pearsonr(cur_df_g2[key1], cur_df_g2[key2])[0],
        corr_tc_daughter_g3=stats.pearsonr(cur_df_g3[key1], cur_df_g3[key2])[0],
        corr_tc_daughter_g23=stats.pearsonr(cur_df_g23[key1], cur_df_g23[key2])[0],
    )

def corr_tc_output(full_df):
    no_gp = (full_df["type_D1"] != "GP") & (full_df["type_D2"] != "GP")
    prog_df = full_df[no_gp].copy()
    prog_df["prog_D1"] = prog_df["type_D1"].apply(lambda x: "Cycling" if x in ["IP", "RG"] else "PM")
    prog_df["prog_D2"] = prog_df["type_D2"].apply(lambda x: "Cycling" if x in ["IP", "RG"] else "PM")
    prog_df["nb_child_pm"] = (prog_df["prog_D1"] == "PM").astype(int) + (prog_df["prog_D2"] == "PM").astype(int)
    prog_df_g2 = prog_df[prog_df["group"].isin([2])]
    prog_df_g3 = prog_df[prog_df["group"].isin([3])]
    prog_df_g23 = prog_df[prog_df["group"].isin([2, 3])]
    return dict(
        corr_tc_output_g2=stats.pearsonr(prog_df_g2["nb_child_pm"], prog_df_g2["Tc_h_M"])[0],
        corr_tc_output_g3=stats.pearsonr(prog_df_g3["nb_child_pm"], prog_df_g3["Tc_h_M"])[0],
        corr_tc_output_g23=stats.pearsonr(prog_df_g23["nb_child_pm"], prog_df_g23["Tc_h_M"])[0],
    )

def fate_corr(full_df):
    kC, kN = "Cycling", "PM"
    prog_df = full_df.copy()
    no_gp = (full_df["type_D1"] != "GP") & (full_df["type_D2"] != "GP")
    filt = prog_df["group"].isin([2, 3]) & no_gp
    prog_df = prog_df[filt]
    prog_df["prog_D1"] = prog_df["type_D1"].apply(lambda x: kC if x in ["IP", "RG"] else kN)
    prog_df["prog_D2"] = prog_df["type_D2"].apply(lambda x: kC if x in ["IP", "RG"] else kN)
    res_fate_cor = prog_df.groupby(["prog_D1", "prog_D2"]).size()
    
    CC, CN, NN = res_fate_cor[kC][kC], res_fate_cor[kC][kN] + res_fate_cor[kN][kC], res_fate_cor[kN][kN]
    T = CC + CN + NN
    pCC, pCN, pNN = CC / T, CN / T, NN / T
    all_C, all_N = 2 * CC + CN, 2 * NN + CN
    pC, pN = all_C / (T * 2), all_N / (T * 2)
    eCC, eCN, eNN = pC**2, 2*pC*pN, pN**2
    F_metric = 1 - pCN / eCN
    return dict(
        F_metric=F_metric,
    )

def preprocess_corr_metrics(param):
    history = preprocess_history(param)
    merged = merge_parents(history)
    res = dict(
        **metrics_tc_mother_daughter(merged),
        **metrics_tc_daughters(merged),
        **corr_tc_output(merged),
        **fate_corr(merged)
    )
    return res

In [236]:
class RendererPlot:
    def __init__(self, name, preprocess_func, mean_func, plot_func):
        self.name = name
        self.preprocess_func = preprocess_func
        self.mean_func = mean_func
        self.plot_func = plot_func
        
    def __call__(self, metagroup, title=None):
        lines = list()
        for group_value, group in metagroup.items():
            ls = []
            for sample in group:
                ls.append(self.preprocess_func(sample))
            line = self.mean_func(ls)
            line["label"] = group_value
            lines.append(line)
        self.plot_func(lines, title=title)

In [264]:
render_nb_progenitor = RendererPlot("nb_progenitor", preprocess_nb_progenitor, mean_sd, plot_lines)
render_nb_cells = RendererPlot("nb_cells", preprocess_nb_cells, mean_sd, plot_lines)
render_ratio = RendererPlot("ratioIP", preprocess_ratio, mean_sd, plot_lines)
render_progeny = RendererPlot("progeny leaves", preprocess_progeny, mean_progeny, plot_sum)
render_progeny_all = RendererPlot("progeny all", preprocess_progeny_all, mean_progeny, plot_sum)
render_corr_metrics = RendererPlot("corr_metrics", preprocess_corr_metrics, mean_dict, plot_metrics)
# render_neighborhood = RendererPlot(preprocess_neighborhood, mean_dict, plot_metrics)

renderers = [render_nb_progenitor, render_nb_cells, render_ratio,
            render_progeny, render_progeny_all, render_corr_metrics]

In [229]:
def get_param_name(key):
    return [x[1] for x in key if x[0] == "param"][0].strip("_")

def get_name_for_file(key):
    return "_".join([x[1].strip("_") for x in key])

In [230]:
get_name_for_file(key)

'expLI_tristateLI_bias_ratio_3'

In [263]:
for key, meta in tqdm(metagroups.items()):
    for renderer in renderers:
        title = " ".join([listtuple_to_dict(key).get("model"), get_param_name(key), renderer.name])
        renderer(meta, title=title)
        plt.savefig(P(ROOT_EXPORT) / (get_name_for_file(key) + "_" + renderer.name + ".png"))
        plt.close()
    break

  0%|          | 0/60 [04:33<?, ?it/s]


# Debug

In [284]:
@loader
def progeny_along_time(history, _type="RG", only_leaves=True, step=2., appear_time=49.):
    ret_df = pd.DataFrame()
    
    min_time, max_time = min(history["appear_time"]), max(history["appear_time"])
    dict_trees = dict()
    # build trees
    for i, row in history.iterrows():
        if row["appear_time"] == appear_time:
            tree = tree_from_cell(history, row["index"])
            dict_trees.update({x.name: x for x in tree.traverse()})
    
    if _type not in history["type"]:
        _type = "Cycling"
    
    for T in np.arange(min_time, max_time, step):
        count_T = Counter()
        df_T = history[(history["appear_time"] >= T) &
                  (history["appear_time"] < (T + step)) & (history["type"] == _type)]
        N = len(df_T)
        print(N)
        if not N:
            continue
        
        for i, cell in df_T.iterrows():
            progeny = count_total_progeny(history, cell["index"], dict_trees.get(cell["index"]), only_leaves=only_leaves)
            count_T.update(progeny)
            
        new_row = {k: v / N for k, v in dict(count_T).items()}
        new_row["time"] = T
        ret_df = ret_df.append(new_row, ignore_index=True)
        
    ret_df = ret_df.fillna(0.0)
        
    return ret_df

progeny_along_time()

85
82
101
121
149
170
183
231
249
246
226
233
217
237
201
195
164
190
157
175
34


,Cycling,PostMitotic,time
0,3.800000,74.564706,49.0
1,2.500000,48.585366,51.0
2,1.514851,31.039604,53.0
3,1.231405,23.008264,55.0
4,1.026846,17.348993,57.0
5,0.870588,15.111765,59.0
6,0.612022,11.628415,61.0
7,0.575758,9.251082,63.0
8,0.481928,7.493976,65.0
9,0.556911,7.426829,67.0


In [277]:
for i, (key, meta) in enumerate(metagroups.items()):
    if i == 1:
        for group_value, group in meta.items():
            ls = []
            for sample in group:
                print(sample)
                # file = "history_" + sample["name"] + ".csv"
                # df = pd.read_csv(P(ROOT) / file, index_col=0)
                x = preprocess_progeny(sample)
                print(x)
                raise

{'exp': 'expLI', 'model': 'basic', 'param': '_diff_values_4', 'value': '0.480', 'n': 'n2', 'name': 'expLI_basic__diff_values_4_0.480_n2'}
progeny_along_time inputs were saved at /home/nathan/other/thesis_nathan/EmbryonicCortexModelling/cbmos/.debug/progeny_along_time !
Empty DataFrame
Columns: []
Index: []


RuntimeError: No active exception to reraise

In [266]:
@loader
def mean_progeny(ls):
    """Runs on all columns"""
    print(ls)
    min_len = min([len(sample["time"]) for sample in ls])
    dict_res = dict(x=ls[0]["time"][:min_len])
    
    col = {colname for sample in ls for colname in sample.columns} - {"time"}

    for c in col:
        mean = np.mean([sample.get(c, [0] * min_len)[:min_len] for sample in ls], axis=0)
        sd = np.std([sample.get(c, [0] * min_len)[:min_len] for sample in ls], axis=0)
        dict_res[c] = {"mean": mean, "sd": sd}
    return dict_res

mean_progeny()

[Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: [], Empty DataFrame
Columns: []
Index: []]


KeyError: 'time'